In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import chess
import chess.engine
from stockfish import Stockfish


In [39]:
# Define la red neuronal convolucional con Dropout
class ChessCNN(nn.Module):
    def __init__(self):
        super(ChessCNN, self).__init__()
        self.conv1 = nn.Conv2d(13, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(p=0.5)  # Agregamos dropout con probabilidad 0.5
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 4096)  # Aproximadamente el número máximo de movimientos posibles

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 256 * 8 * 8)
        x = self.dropout(torch.relu(self.fc1(x)))  # Aplicamos dropout
        return self.fc2(x)


In [40]:
# Función para convertir el tablero en una matriz para la red neuronal
def board_to_matrix(board):
    matrix = np.zeros((13, 8, 8))
    piece_map = board.piece_map()

    # Piezas en el tablero
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[piece_type + piece_color, row, col] = 1

    # Movimientos legales
    legal_moves = board.legal_moves
    for move in legal_moves:
        row_to, col_to = divmod(move.to_square, 8)
        matrix[12, row_to, col_to] = 1

    return matrix


In [41]:
# Crea un dataset para la red neuronal
def create_input_for_nn(board):
    X = board_to_matrix(board)
    return torch.tensor(X, dtype=torch.float32).unsqueeze(0)


In [42]:
# Sistema de recompensas para aprendizaje por refuerzo
def calculate_reward(board):
    if board.is_checkmate():
        if board.turn == chess.WHITE:
            return 1  # IA gana
        else:
            return -1  # IA pierde
    elif board.is_stalemate() or board.is_insufficient_material():
        return 0  # Empate
    else:
        return 0  # Ningún evento significativo


In [43]:
def train_against_stockfish(model, optimizer, stockfish_path, epochs=10):
    stockfish = Stockfish(stockfish_path)
    stockfish.set_depth(15)

    for epoch in range(epochs):
        board = chess.Board()
        game_over = False
        winner = None
        
        while not game_over:
            # IA predice el siguiente movimiento
            model.train()  # Asegúrate de que el modelo está en modo de entrenamiento
            board_matrix = create_input_for_nn(board)
            move_probabilities = model(board_matrix).squeeze()
            
            # Obtener la lista de movimientos legales
            legal_moves = list(board.legal_moves)
            
            # Crear una lista para almacenar las probabilidades de los movimientos legales
            legal_move_probs = [move_probabilities[i].item() for i, move in enumerate(legal_moves)]
            legal_move_indices = list(range(len(legal_moves)))
            
            # Convertir las probabilidades legales en tensor
            legal_move_probs = torch.tensor(legal_move_probs)
            
            # Seleccionar el movimiento legal con la mayor probabilidad
            if len(legal_move_probs) > 0:
                chosen_move_idx = torch.argmax(legal_move_probs).item()
                chosen_move = legal_moves[legal_move_indices[chosen_move_idx]]
                board.push(chosen_move)
            else:
                print("No hay movimientos legales disponibles.")
                break

            # Comprobar si el juego ha terminado
            if board.is_game_over():
                game_over = True
                winner = 'IA' if calculate_reward(board) == 1 else 'Stockfish' if calculate_reward(board) == -1 else 'Empate'
                break

            # Stockfish juega su mejor movimiento
            stockfish.set_fen_position(board.fen())
            best_move = stockfish.get_best_move()
            board.push(chess.Move.from_uci(best_move))

            if board.is_game_over():
                game_over = True
                winner = 'IA' if calculate_reward(board) == 1 else 'Stockfish' if calculate_reward(board) == -1 else 'Empate'
                break

            # Calcular recompensa y actualizar modelo
            reward = calculate_reward(board)
            optimizer.zero_grad()
            loss = -torch.log(move_probabilities[chosen_move_idx]) * reward
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs} terminado. Ganador: {winner}")

    # Elimina esta línea si no hay método quit()
    # stockfish.quit()


In [45]:
# Definir el modelo, optimizador y entrenar contra Stockfish
if __name__ == "__main__":
    # Ruta a tu ejecutable de Stockfish
    stockfish_path = "stockfish-windows-x86-64-avx2.exe"

    model = ChessCNN()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Entrenamiento contra Stockfish
    train_against_stockfish(model, optimizer, stockfish_path, epochs=5)


Epoch 1/5 terminado. Ganador: IA
Epoch 2/5 terminado. Ganador: IA


KeyboardInterrupt: 